In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 3GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=3072)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [16]:
"""
def normalize_img(image, label):
  Normalizes images: `uint8` -> `float32`.
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)
"""

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def preprocess_a(data, batch_size):
    # image should be float
    data = data.map(lambda x, t: (tf.cast(x, float), t))
    # image should be flattened
    data = data.map(lambda x, t: (tf.reshape(x, (-1,)), t))
    # image vector will here have values between -1 and 1
    data = data.map(lambda x,t: ((x/128.)-1., t))
    # we want to have two mnist images in each example
    # this leads to a single example being ((x1,y1),(x2,y2))
    zipped_ds = tf.data.Dataset.zip((data.shuffle(2000), 
                                     data.shuffle(2000)))
    # map ((x1,y1),(x2,y2)) to (x1,x2, y1==y2*) *x1[1] + x2[1] >= 5
    zipped_ds = zipped_ds.map(lambda x1, x2: (x1[0], x2[0], x1[1] + x2[1] >= 5))
    # transform boolean target to int
    zipped_ds = zipped_ds.map(lambda x1, x2, t: (x1,x2, tf.cast(t, tf.int32)))
    # batch the dataset
    zipped_ds = zipped_ds.batch(batch_size)
    # prefetch
    zipped_ds = zipped_ds.prefetch(tf.data.AUTOTUNE)
    return zipped_ds

def preprocess_b(data, batch_size):
    # image should be float
    data = data.map(lambda x, t: (tf.cast(x, float), t))
    # image should be flattened
    data = data.map(lambda x, t: (tf.reshape(x, (-1,)), t))
    # image vector will here have values between -1 and 1
    data = data.map(lambda x,t: ((x/128.)-1., t))
    # we want to have two mnist images in each example
    # this leads to a single example being ((x1,y1),(x2,y2))
    zipped_ds = tf.data.Dataset.zip((data.shuffle(2000), 
                                     data.shuffle(2000)))
    # map ((x1,y1),(x2,y2)) to (x1,x2, y1==y2*) *x1[1] + x2[1] >= 5
    zipped_ds = zipped_ds.map(lambda x1, x2: (x1[0], x2[0], x1[1] - x2[2]))
    # transform boolean target to int
    zipped_ds = zipped_ds.map(lambda x1, x2, t: (x1,x2, tf.cast(t, tf.int32)))
    # batch the dataset
    zipped_ds = zipped_ds.batch(batch_size)
    # prefetch
    zipped_ds = zipped_ds.prefetch(tf.data.AUTOTUNE)
    return zipped_ds
    

train_ds_a = preprocess_a(ds_train, batch_size=128) #train_ds.apply(preprocess)
train_ds_b = preprocess_b(ds_train, batch_size=128) #train_ds.apply(preprocess)
val_ds_a = preprocess_a(ds_test, batch_size=128) #val_ds.apply(preprocess)

val_ds_b = preprocess_b(ds_test, batch_size=128) #val_ds.apply(preprocess)

StagingError: in user code:

    File "C:\Users\Benny\AppData\Local\Temp\ipykernel_2728\2726633912.py", line 61, in None  *
        lambda x1, x2: (x1[0], x2[0], x1[1] - x2[2])

    IndexError: tuple index out of range


In [12]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test,
)


Epoch 1/20
1875/1875 [==============================] - 15s 7ms/step - loss: 0.2562 - sparse_categorical_accuracy: 0.9261 - val_loss: 0.1315 - val_sparse_categorical_accuracy: 0.9613
Epoch 2/20
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1138 - sparse_categorical_accuracy: 0.9656 - val_loss: 0.0989 - val_sparse_categorical_accuracy: 0.9700
Epoch 3/20
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0788 - sparse_categorical_accuracy: 0.9759 - val_loss: 0.0838 - val_sparse_categorical_accuracy: 0.9739
Epoch 4/20
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0586 - sparse_categorical_accuracy: 0.9820 - val_loss: 0.0826 - val_sparse_categorical_accuracy: 0.9732
Epoch 5/20
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0465 - sparse_categorical_accuracy: 0.9859 - val_loss: 0.0801 - val_sparse_categorical_accuracy: 0.9750
Epoch 6/20
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0359 -

In [ ]:
Epoch 1/6
469/469 [==============================] - 6s 7ms/step - loss: 0.3664 - sparse_categorical_accuracy: 0.8992 - val_loss: 0.2011 - val_sparse_categorical_accuracy: 0.9396
Epoch 2/6
469/469 [==============================] - 3s 6ms/step - loss: 0.1666 - sparse_categorical_accuracy: 0.9523 - val_loss: 0.1393 - val_sparse_categorical_accuracy: 0.9604
Epoch 3/6
469/469 [==============================] - 3s 6ms/step - loss: 0.1186 - sparse_categorical_accuracy: 0.9661 - val_loss: 0.1165 - val_sparse_categorical_accuracy: 0.9656
Epoch 4/6
469/469 [==============================] - 3s 6ms/step - loss: 0.0926 - sparse_categorical_accuracy: 0.9733 - val_loss: 0.0980 - val_sparse_categorical_accuracy: 0.9708
Epoch 5/6
469/469 [==============================] - 3s 6ms/step - loss: 0.0754 - sparse_categorical_accuracy: 0.9786 - val_loss: 0.0877 - val_sparse_categorical_accuracy: 0.9731
Epoch 6/6
469/469 [==============================] - 3s 6ms/step - loss: 0.0621 - sparse_categorical_accuracy: 0.9820 - val_loss: 0.0825 - val_sparse_categorical_accuracy: 0.9741